<a href="https://colab.research.google.com/github/PrishaAggarwal/Conversational_AI_All_cources/blob/main/Accelerated_Data_Science/Lab/My_files_Colab/Convo_AI_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Feature Selection

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('/content/weather - weather.csv')
df

,Outlook,Temp,Humidity,Windy,Play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rainy,mild,high,weak,yes
4,rainy,cool,normal,weak,yes
5,rainy,cool,normal,strong,no
6,overcast,cool,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cool,normal,weak,yes
9,rainy,mild,normal,weak,yes


In [ ]:
def chi_sq_value(df,attribute,target):
  #chi sq score we find between attribute and target
  #phele find attributes ki unique values
  attr_values=df[attribute].unique()
  #target ki unique values
  target_values=df[target].unique()
  chi_sq = 0
  #har pair of attribute and target value upar wale variable mein dalni hai
  for attr_value in attr_values:
    for target_value in target_values:
      #finding how many yes and how many no in each attribute with target
      observed_freq =  len(df[(df[attribute] == attr_value) & (df[target] == target_value)])
      #we are finding voh rows jismein ek particular attribute ke under target value is equal to what we are finding
      #for exmaple gender is attribute and target is yes
      #we want how many rows of such things so we will find the length of above

      #expected freq mein we assume the events are independent
      prob_attribute = len(df[df[attribute] == attr_value]) / len(df) #divided by length to find the probability
      #len(df) is total number of rows
      prob_target = len(df[df[target] == target_value]) / len(df)
      expected_freq = prob_attribute * prob_target * len(df)
      chi_sq = (observed_freq - expected_freq) ** 2 / expected_freq

  #har combination ka aisa chi_sq score aa jayega using above 2 loops
  return chi_sq

In [ ]:
chi_sq_value(df,'Outlook','Play')
#similarly har feature ke nikalkr onko store kr lenge and increasing mein kr denge and max wale utha lenge...done after preprocessed

0.01428571428571436

In [ ]:
df1=pd.read_csv('/content/weather - weather.csv')
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df1['Outlook']=le.fit_transform(df1['Outlook'])
df1['Temp']=le.fit_transform(df1['Temp'])
df1['Humidity']=le.fit_transform(df1['Humidity'])
df1['Windy'] = le.fit_transform(df1['Humidity'])
#categorial values converted into numerical values
df1

,Outlook,Temp,Humidity,Windy,Play
0,2,1,0,0,no
1,2,1,0,0,no
2,0,1,0,0,yes
3,1,2,0,0,yes
4,1,0,1,1,yes
5,1,0,1,1,no
6,0,0,1,1,yes
7,2,2,0,0,no
8,2,0,1,1,yes
9,1,2,1,1,yes


In [ ]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

#creating object
ss=SelectKBest(chi2,k=2) #we have to give kitne best features rakhne hai i.e ka
#chi2 giving to tell what function we wanna apply i.e chi_sq
#automatically it is applying information gain
#inbuilt wala expects ki sab columns already numerical hai
ss.fit(df1.iloc[:,:-1],df1.iloc[:,-1])
#if we use here fit_transform instead of fit then voh bas imp features rakhega and baaki remove krke dataframe de dega

SelectKBest(k=2, score_func=<function chi2 at 0x7815950e9800>)

In [ ]:
ss.get_support() #tells you which feature to keep and which not to
#outlook and humidity more imp since they have more chi sq score
#in built method ka chi sq is normalized...features same honge but value wont match
#sb chi sq score ki variance niklakr osse divide kr dete hia har chi sq value

array([ True, False, False,  True])

In [ ]:
def chi_sq_selection(df,k):
  attributes = df.columns[:-1] #last column chodkr sab columns lelo
  #target wala column is last column
  target = df.columns[-1]
  chi_sq = {} #dictionary to store chi sq of all attributes
  for attributes in attributes:
    #ek ek krke har attribute pick krenge to find its chi sq score
    chi_sq[attributes] = chi_sq_value(df,attributes,target) #har attribute ka computer chi sq using the function we created
    #lets give here also a k
    #sort krlo
  sorted_values = sorted(chi_sq.items(),key=lambda x:x[1],reverse=True) #items of dictionary ko sort krna hai
  #lambda wala says hum har key pair value uthayenge and values ke acc sort kroge i.e x[1]
  #by deafult in ascending order toh reserved true
  selected_features = [f for f,_ in sorted_values[:k]] #hume value nhi chahiye...we just want the feature toh bas pheli k values uthakr select krli
  return selected_features


In [ ]:
chi_sq_selection(df,2)

['Humidity', 'Windy']

In [ ]:
#INFORMATION GAIN

In [ ]:
def find_entropy(df):
  target = df.columns[-1] #usually last column is target toh vhi lenge'
  target_values = df[target].unique()
  entropy = 0 #ismein har class ki entropy we will add
  for target_value in target_values:
    #suppose yes and no
    #toh dono ki probability we will find
    prob = len(df[df[target] == target_value])/len(df)
    entropy += -prob*np.log2(prob)
  return entropy

In [ ]:
find_entropy(df) #pure sample ki entropy we have found

np.float64(0.9402859586706311)

In [ ]:
#aab har attribute ki entropy we will find
#suppose ek attribute ki unique values nikali and then further entopy ke formula se we will the entropy
def find_avg_info_entropy(df,attribute): #we are passing attribute since ek attribute ki avg entropy niklegi
  attr_values=df[attribute].unique()
  target=df.columns[-1]
  target_values = df[target].unique()
  avg_info_entropy = 0
  for attr_value in attr_values: #ek ek krke har attribute ki unique we will consider
    entropy_subsample = 0 #iss unique value under attirbute we will find entropy...suppose sunny in outllook
    #now we will find sunny ke andar kitne yes hia and kitne no hai
    for target_value in target_values:
      num = len(df[(df[attribute] == attr_value) & (df[target] == target_value)]) #seeing kitni rows mein sunny and yes hai...this is numerator
      den = len(df[df[attribute] == attr_value]) #total sunny ki rows..yeh denominator
      prob = num/den
      entropy_subsample += prob*np.log2(prob+1e-7) #here added a small value 1e-7 so that by chance 0 aaya and log 0 is not defined so osko avoid krne ke liye
    avg_info_entropy += (len(df[df[attribute]==attr_value])/len(df))*(entropy_subsample) #iss entropy ko iske weight se dividing
  return avg_info_entropy

In [ ]:
find_avg_info_entropy(df,'Outlook')

np.float64(-0.6935358915770655)

In [ ]:
#again selecting k best features on basis of information gain
def IG_selection(df,k):
  attributes=df.columns[:-1] #all except last
  target = df.columns[-1]
  #har attibute ka info gain and jiska best we have to choose that
  IG={}
  for attribute in attributes:
    IG[attribute]=find_entropy(df) - find_avg_info_entropy(df,attribute)
  sorted_values=sorted(IG.items(),key=lambda x:x[1],reverse=True)
  selected_features=[f for f,_ in sorted_values[:k]] #first k values we take
  return selected_features

In [ ]:
IG_selection(df,2)

['Temp', 'Windy']

In [ ]:
#inbuilt method in IG
#but yeh inbuilt method computes gain ratio not IG
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

#yeh bhi numerical values expects toh df1 we will use
ss=SelectKBest(mutual_info_classif,k=2)
ss.fit(df1.iloc[:,:-1],df1.iloc[:,-1])

SelectKBest(k=2, score_func=<function mutual_info_classif at 0x781595557060>)

In [ ]:
ss.get_support()

array([ True, False, False,  True])